# Algebraic Prolongation Examples with `dgcv`
<div style="display: flex; justify-content: space-between; align-items: center; font-size: smaller;">
    <div style="text-align: right; white-space: nowrap;">
    </div>
    <div style="flex: 0 0 80%; margin: 0 auto; text-align: center;">
        <em>
            Tutorial examples for the <code>Tanaka_symbol</code> class and <code>Tanaka_symbol.prolong()</code> method
        </em>
    </div>
    <div style="text-align: right; white-space: nowrap; padding: 0 0.5em; border-left: 1px solid;">
    by David Sykes
    </div>
</div>

---

This notebook contains examples of algebraic prolongations computed using the `Tanaka_symbol` class in `dgcv`.

The examples illustrate several ways to initialize algebraic symbol data, including:

- Direct instantiation from various types of defining data, such as structure equations, structure coefficients, vector field representations, or matrix representations
- Derivation from regular vector distributions defined using the `dgcv.distribution` class

Additionally, the examples demonstrate optional features of the `Tanaka_symbol.prolong()` method, such as:

- Prolongation of subspaces (not necessarily subalgebras) within Lie algebras
- Prolongations subject to various additional constraints

## Preamble (package imports and notebook settings)
###### requires dgcv version ≥ v0.3.3

In [1]:
from dgcv import *
set_dgcv_settings(format_displays=True,
                  use_latex=True,
                  ask_before_overwriting_objects_in_vmf=False,
                  forgo_warnings=True)

# Example 1: Obtaining symbol data from a distribution

In this example, we
1. begin with a regular vector distribution,
2. compute its nilpotent approximation at a point,
3. cast the nilpotent approximation into a `Tanaka_symbol` class object,
4. compute its prolongations, and
5. convert the computed maximal prolongation into an instance of the `algebra_class` class (`dgcv`'s abstract representation of finite-dimensional algebras).

We will initialize a distribution by specifying a set of vector fields that span it, and give this list to `distribution` class initializer.

For this example, we will set up a rank 2 distribution `D` on $\mathbb{R}^5$.

In [2]:
createVariables('x', 5, withVF=True)            # initializes coordinates x1, x2, ..., with corresponding coordinate VF D_x1, D_x2, ...
X1 = D_x1                                       # labels a vector field X1
X2 = -2*D_x2+x1*D_x3+x1**2*D_x4/2+x1*x2*D_x5    # labels another vector field X2
D = distribution([X1,X2])                       # initializes distribution spanned by X1 and X2
show(f'We just initialized two vector fields $$X_1={LaTeX(X1)} \\quad\\text{{and}}\\quad X_2={LaTeX(X2)}$$ and a distribution `D` defined by their span.')

<IPython.core.display.Latex object>

The `distribution` class has methods for computing properties of a distribution such as `distribution.derived_flag()` and `distribution.nilpotent_approximation()` for computing derived flags and nilpotent approximations at specified points. The latter method returns an `algebra_class` object, which is the `dgcv` class representing finite dimensional algebras.

In [3]:
D.nilpotent_approximation(label='g',basis_labels=['e1','e2','e3','e4','e5'])     # by default the nipotent approximation algebra is computed at the origin, but this can be adjusted with the `expansion_point` keyword

This just created a graded nilpotent Lie algebra labeled `g`. It was assigned weights corresponding to the distribution's derived flag levels. Here is its multiplication table:

In [4]:
g.multiplication_table()

,$e_{1}$,$e_{2}$,$e_{3}$,$e_{4}$,$e_{5}$
$e_{1}$,$0$,$-e_{3}$,$-e_{4}$,$0$,$0$
$e_{2}$,$e_{3}$,$0$,$-e_{5}$,$0$,$0$
$e_{3}$,$e_{4}$,$e_{5}$,$0$,$0$,$0$
$e_{4}$,$0$,$0$,$0$,$0$,$0$
$e_{5}$,$0$,$0$,$0$,$0$,$0$


To see an algebra class object's assigned gradings (if there are any), use the `algebra_class.grading` attribute to retrieve a list of assigned grading vectors. Any weight vector can be assigned to an algebra class object, so having assigned gradings does not imply the gradings are compatible with the algebra structure. Check this with the class method `algebra_class.check_grading_compatibility`, which returns `True` if it is compatible and false otherwise:

In [5]:
if g.check_grading_compatibility() is True:
    show(f'The Lie algebra ${LaTeX(g)}$ is graded with respect to grading weights {g.grading}, i.e. its basis vectors have weights {LaTeX_eqn_system(dict(zip(g.basis,g.grading[0])),left_prefix='[',left_suffix=r']_{\text{wt}}',one_line=True,add_period=True)}')
else:
    show('The Lie algebra ${LaTeX(g)}$ is not compatible with its assigned grading.')

<IPython.core.display.Latex object>

As a fundamental graded Lie algebra, we may compute prolongations of $\mathfrak{g}$. For this we convert it to a `Tanaka_symbol` class object by simply passing `g` into the class initializer `Tanaka_symbol()`. Then we can use the `Tanaka_symbol.prolongation(return_symbol=True)` method to compute prolongations. Setting the keyword `return_symbol=True` returns another `Tanaka_symbol` class object (whereas, without setting it a lighter-weight dictionary-like structure containing lists of elements in the prolong would be returned):

In [6]:
g_minus = Tanaka_symbol(g)
g_prolonged = g_minus.prolong(5,return_symbol=True)
g_prolonged.summary()       # outputs a summary table of the prolongation computed on the previous line

Weight,Dimension,Basis
-3,2,"$e_{4}$, $e_{5}$"
-2,1,$e_{3}$
-1,2,"$e_{1}$, $e_{2}$"
0,4,"$2 e_{4}\otimes e_{4}^* + e_{5}\otimes e_{5}^* + e_{3}\otimes e_{3}^* + e_{1}\otimes e_{1}^*$, $e_{4}\otimes e_{5}^* + e_{1}\otimes e_{2}^*$, $e_{4}\otimes e_{4}^* + 2 e_{5}\otimes e_{5}^* + e_{3}\otimes e_{3}^* + e_{2}\otimes e_{2}^*$, $e_{5}\otimes e_{4}^* + e_{2}\otimes e_{1}^*$"
1,2,output too long to display; raise `display_length` to a higher bound if needed.
2,1,output too long to display; raise `display_length` to a higher bound if needed.
3,2,output too long to display; raise `display_length` to a higher bound if needed.


The summary table above only shows the weighted levels of the computed prolongation. It only displays 4 non-negatively weighted levels despite our asking it to compute 5. This implies the algorithm found that higher prolongations beyond wieght 3 are trivial, and thus the prolongation displayed above is complete.

To extract the abstract algebra structure from the prolongation we just computed, we can pass it to the `createAlgebra` function, which will instantiate an `algebra_class` object with the prolongation's structure and assigned label of our choosing. Let's use the label `prolongation_algebra` for this example:

In [7]:
createAlgebra(g_prolonged,label='prolongation_algebra',basis_labels='eta')

Use the `multiplication_table` method to see the algebra's multiplication table:

In [8]:
prolongation_algebra.multiplication_table()

,$\eta_{1}$,$\eta_{2}$,$\eta_{3}$,$\eta_{4}$,$\eta_{5}$,$\eta_{6}$,$\eta_{7}$,$\eta_{8}$,$\eta_{9}$,$\eta_{10}$,$\eta_{11}$,$\eta_{12}$,$\eta_{13}$,$\eta_{14}$
$\eta_{1}$,$0$,$0$,$0$,$0$,$0$,$-2 \cdot \eta_{1}$,$0$,$-\eta_{1}$,$-\eta_{2}$,$0$,$-\eta_{3}$,$\eta_{4}$,$\eta_{7}$,$\eta_{6}$
$\eta_{2}$,$0$,$0$,$0$,$0$,$0$,$-\eta_{2}$,$-\eta_{1}$,$-2 \cdot \eta_{2}$,$0$,$- \frac{3}{2} \cdot \eta_{3}$,$0$,$\eta_{5}$,$\eta_{8}$,$\eta_{9}$
$\eta_{3}$,$0$,$0$,$0$,$\eta_{1}$,$\eta_{2}$,$-\eta_{3}$,$0$,$-\eta_{3}$,$0$,$2 \cdot \eta_{4}$,$- \frac{4}{3} \cdot \eta_{5}$,$- \frac{1}{3} \cdot \eta_{6} - \frac{1}{3} \cdot \eta_{8}$,$\frac{2}{3} \cdot \eta_{10}$,$\eta_{11}$
$\eta_{4}$,$0$,$0$,$-\eta_{1}$,$0$,$-\eta_{3}$,$-\eta_{4}$,$0$,$0$,$-\eta_{5}$,$- \frac{3}{2} \cdot \eta_{7}$,$- \frac{2}{3} \cdot \eta_{6} + \frac{1}{3} \cdot \eta_{8}$,$\frac{2}{3} \cdot \eta_{10}$,$0$,$-\eta_{12}$
$\eta_{5}$,$0$,$0$,$-\eta_{2}$,$\eta_{3}$,$0$,$0$,$-\eta_{4}$,$-\eta_{5}$,$0$,$\frac{1}{2} \cdot \eta_{6} - \eta_{8}$,$-\eta_{9}$,$-\eta_{11}$,$-\eta_{12}$,$0$
$\eta_{6}$,$2 \cdot \eta_{1}$,$\eta_{2}$,$\eta_{3}$,$\eta_{4}$,$0$,$0$,$\eta_{7}$,$0$,$-\eta_{9}$,$0$,$-\eta_{11}$,$-\eta_{12}$,$-\eta_{13}$,$-2 \cdot \eta_{14}$
$\eta_{7}$,$0$,$\eta_{1}$,$0$,$0$,$\eta_{4}$,$-\eta_{7}$,$0$,$\eta_{7}$,$\eta_{6} - \eta_{8}$,$0$,$- \frac{2}{3} \cdot \eta_{10}$,$0$,$0$,$-\eta_{13}$
$\eta_{8}$,$\eta_{1}$,$2 \cdot \eta_{2}$,$\eta_{3}$,$0$,$\eta_{5}$,$0$,$-\eta_{7}$,$0$,$\eta_{9}$,$-\eta_{10}$,$0$,$-\eta_{12}$,$-2 \cdot \eta_{13}$,$-\eta_{14}$
$\eta_{9}$,$\eta_{2}$,$0$,$0$,$\eta_{5}$,$0$,$\eta_{9}$,$-\eta_{6} + \eta_{8}$,$-\eta_{9}$,$0$,$- \frac{3}{2} \cdot \eta_{11}$,$0$,$0$,$-\eta_{14}$,$0$
$\eta_{10}$,$0$,$\frac{3}{2} \cdot \eta_{3}$,$-2 \cdot \eta_{4}$,$\frac{3}{2} \cdot \eta_{7}$,$- \frac{1}{2} \cdot \eta_{6} + \eta_{8}$,$0$,$0$,$\eta_{10}$,$\frac{3}{2} \cdot \eta_{11}$,$0$,$2 \cdot \eta_{12}$,$\frac{3}{2} \cdot \eta_{13}$,$0$,$0$


The `algebra_class` class has built-in methods for computing basic an algebra's properties (e.g., skew symmetry, Jacobi identities, solvability, etc.). 

For example, our `prolongation_algebra` happens to be semi-simple, and we can confirm that using a class method. By setting `verbose=True`, the algorithm will report progress updates as it carries our the computation:

In [9]:
prolongation_algebra.is_semisimple(verbose=True,return_bool=False)

prolongation_algebra is skew-symmetric.
prolongation_algebra satisfies the Jacobi identity.
prolongation_algebra is a Lie algebra.
Progress update: computing determinant of the Killing form...
prolongation_algebra is semisimple.


This example's prolongation is the exceptional simple algebra $\mathfrak{g}_2$ coming from a flat (2,3,5) distribution.

# Example 2: Initializing symbol data from structure equations

In Example 2 and Example 3 we will study symbol data that is a special case of the general symbols whose prolongations are described in Theorem 5.3 of

> **Porter, C. and Zelenko, I.**
> *Absolute parallelism for 2-nondegenerate CR structures via bigraded Tanaka prolongation*  
> J. reine angew. Math. (Crelles Journal) 777 (2021), 195–250

First we will compute part of a standard prolongation that is infinite-dimensional. Then Example 3 demonstrations computations of finite-dimensional reduced prolongations.

Let's initialize a graded Lie algebra by providing structure equations, a basis, and grading directly. We will use the `createAlgebra` function:

In [10]:
createVariables('a',11,initialIndex=0)      # initializing variables a0,...,a6, w.r.t. which we'll 
                                            # write structure equations
structure_eqns = {(a1,a4):a0,(a2,a3):a0,(a5,a1):a4,(a6,a3):a2,(a7,a1):a1,(a7,a2):a2,(a7,a3):-a3,(a7,a4):-a4,(a7,a5):-2*a5,(a7,a6):2*a6,(a8,a0):2*a0,(a8,a1):a1,(a8,a2):a2,(a8,a3):a3,(a8,a4):a4,(a9,a1):a1,(a9,a2):-a2,(a9,a3):a3,(a9,a4):-a4,(a9,a5):-2*a5,(a9,a6):-2*a6,(a10,a1):a2,(a10,a3):-a4,(a10,a9):2*a10}
createAlgebra(structure_eqns,     # algebra data (many formats supported)
                'g',                # label for the algebra
                basis_labels=[f'e{j}' for j in range(11)],   # label for the algebras basis elements (optional)
                grading=[-2,-1,-1,-1,-1,0,0,0,0,0,0],   # weights assigned to basis elements (determines gradings)
                assume_Lie_alg=True,    # completes structure data to be skew-symmetric if not explicitly supplied
                basis_order_for_supplied_str_eqns=a     # this is only relevant when initializing from structure equations data. This optional argument let's us a specify an order for the basis extracted from the structure equations. In this example we are telling it to use the ordering in a=(a0,a1,...,a6)
                )
g.multiplication_table()

,$e_{0}$,$e_{1}$,$e_{2}$,$e_{3}$,$e_{4}$,$e_{5}$,$e_{6}$,$e_{7}$,$e_{8}$,$e_{9}$,$e_{10}$
$e_{0}$,$0$,$0$,$0$,$0$,$0$,$0$,$0$,$0$,$-2 \cdot e_{0}$,$0$,$0$
$e_{1}$,$0$,$0$,$0$,$0$,$e_{0}$,$-e_{4}$,$0$,$-e_{1}$,$-e_{1}$,$-e_{1}$,$-e_{2}$
$e_{2}$,$0$,$0$,$0$,$e_{0}$,$0$,$0$,$0$,$-e_{2}$,$-e_{2}$,$e_{2}$,$0$
$e_{3}$,$0$,$0$,$-e_{0}$,$0$,$0$,$0$,$-e_{2}$,$e_{3}$,$-e_{3}$,$-e_{3}$,$e_{4}$
$e_{4}$,$0$,$-e_{0}$,$0$,$0$,$0$,$0$,$0$,$e_{4}$,$-e_{4}$,$e_{4}$,$0$
$e_{5}$,$0$,$e_{4}$,$0$,$0$,$0$,$0$,$0$,$2 \cdot e_{5}$,$0$,$2 \cdot e_{5}$,$0$
$e_{6}$,$0$,$0$,$0$,$e_{2}$,$0$,$0$,$0$,$-2 \cdot e_{6}$,$0$,$2 \cdot e_{6}$,$0$
$e_{7}$,$0$,$e_{1}$,$e_{2}$,$-e_{3}$,$-e_{4}$,$-2 \cdot e_{5}$,$2 \cdot e_{6}$,$0$,$0$,$0$,$0$
$e_{8}$,$2 \cdot e_{0}$,$e_{1}$,$e_{2}$,$e_{3}$,$e_{4}$,$0$,$0$,$0$,$0$,$0$,$0$
$e_{9}$,$0$,$e_{1}$,$-e_{2}$,$e_{3}$,$-e_{4}$,$-2 \cdot e_{5}$,$-2 \cdot e_{6}$,$0$,$0$,$0$,$-2 \cdot e_{10}$


Having manually supplied structure equations, there is no assurance the initialized `algebra_class` object `g` represents a Lie algebra. To check for Lie algebra properties, we can use the `is_lie_algebra` class method. Including a keyword `verbose=True` returns a detailed report of the checks as they occur:

In [11]:
g.is_lie_algebra(verbose=True,return_bool=False)    # these optional argument make it just print 
                                                    # a report of the Lie algebra test findings.

g is skew-symmetric.
g satisfies the Jacobi identity.
g is a Lie algebra.


Now that we initialized a graded Lie algebra $\mathfrak{g}$ we can pass it to the `Tanaka_symbol` initializer, and start computing prolongations, like in Example 1. It's prolongations will be infinite-dimensional, however, because there are rank 1 elements in degree 0.



In [12]:
symbol_data = Tanaka_symbol(g)
symbol_data.summary()

Weight,Dimension,Basis
-2,1,$e_{0}$
-1,4,"$e_{1}$, $e_{2}$, $e_{3}$, $e_{4}$"
0,6,"$e_{4}\otimes e_{1}^*$, $e_{2}\otimes e_{3}^*$, $e_{1}\otimes e_{1}^* + e_{2}\otimes e_{2}^*- e_{3}\otimes e_{3}^*- e_{4}\otimes e_{4}^*$, $2 e_{0}\otimes e_{0}^* + e_{1}\otimes e_{1}^* + e_{2}\otimes e_{2}^* + e_{3}\otimes e_{3}^* + e_{4}\otimes e_{4}^*$, $e_{1}\otimes e_{1}^*- e_{2}\otimes e_{2}^* + e_{3}\otimes e_{3}^*- e_{4}\otimes e_{4}^*$, $e_{2}\otimes e_{1}^*- e_{4}\otimes e_{3}^*$"


Note from the summary table above, when we cast $\mathfrak{g}$ to a Tanaka_symbol class object, its degree zero component was automatically coverted into homomorphisms acting on the negatively graded components.

As we compute prolongations of this symbol, we will set `report_progress=True` which will output a summary of the prolongation steps and dimension counts obtained as the procedure runs. For this example we will not do anything further with the prolongation, we will set the optional keyword `report_progress_and_return_nothing=True`.

In [13]:
symbol_data.prolong(3,report_progress_and_return_nothing=True)

After 1st iteration:
┌────────────┬───────────────────┐
│ Weights    │ -2 │ -1 │ 0  │ 1  │
├────────────┼───────────────────┤
│ Dimensions │ 1  │ 4  │ 6  │ 8  │
└────────────┴───────────────────┘
After 2nd iteration:
┌────────────┬────────────────────────┐
│ Weights    │ -2 │ -1 │ 0  │ 1  │ 2  │
├────────────┼────────────────────────┤
│ Dimensions │ 1  │ 4  │ 6  │ 8  │ 11 │
└────────────┴────────────────────────┘
After 3rd iteration:
┌────────────┬─────────────────────────────┐
│ Weights    │ -2 │ -1 │ 0  │ 1  │ 2  │ 3  │
├────────────┼─────────────────────────────┤
│ Dimensions │ 1  │ 4  │ 6  │ 8  │ 11 │ 14 │
└────────────┴─────────────────────────────┘


# Example 3: Prolongations that preserve distinguished subspaces

Let's continue from Example 2, so run the example 2 notebook section first. We will compute special reductions of the standard algebraic prolongation defined as follows.

## Theory notes for Example 3:


### prolongation respecting subspaces

For an infinite-dimensional standard prolongation, Imposing further constraints on the prolongation can yield a finite *reduced prolongation*. One such source of constraints is marking distinguished subspaces that the prolongation needs to respect. 

I.e., if we mark a subspace 
$$
S\subset \mathfrak{g},
$$
then in the universal prolongation 
$$
\mathfrak{u}(\mathfrak{g})=\mathfrak{g}+\bigoplus_{\text{higher weights}}\mathfrak{u}_j
$$
there is the maximal subspace 
$$
\mathfrak{u}^\prime(\mathfrak{g},S)=\mathfrak{g}+\bigoplus_{\text{higher weights}}\mathfrak{u}_j^\prime\subset \mathfrak{u}(\mathfrak{g})
$$ 
such that 
$$
v\in S,\, w\in \mathfrak{u}_j^\prime, [v,w]\subset \mathfrak{g}\implies [v,w]\subset S.
$$
This subspace $\mathfrak{u}^\prime(\mathfrak{g},S)$ is the *universal prolongation respecting $S$*. 


### prolongation respecting subspaces reduced by characteristic subspaces
Reductions of the form $\mathfrak{u}^\prime(\mathfrak{g},S)$ need not be subalgebras. In particular $[\mathfrak{u}^\prime_j,\mathfrak{u}^\prime_j]$ need not be contained in $\mathfrak{u}^\prime_j$.

In the algorithm for computing all $\mathfrak{u}^\prime_j$ iteratively, we can reduce $\mathfrak{u}^\prime_j$ by replaceding it with the subspace
$$
\mathfrak{u}_j^{\text{red}}=\{\mathfrak{u}^\prime_j\,|\, [v,\mathfrak{u}^\prime_0]\in \mathfrak{u}^\prime_j\}.
$$
This reduction can be repeated until it stabelizes, i.e., next we may replace $\mathfrak{u}_j^{\text{red}}$ with the smaller subspace
$$
\left(\mathfrak{u}_j^{\text{red}}\right)^{\text{red}}=\{\mathfrak{u}_j^{\text{red}}\,|\, [v,\mathfrak{u}^\prime_0]\in \mathfrak{u}_j^{\text{red}}\},
$$
and repeat this until $\mathfrak{u}_j^{\text{red}}=\left(\mathfrak{u}_j^{\text{red}}\right)^{\text{red}}$.

Computing the maximally reduced $\mathfrak{u}_j^{\text{red}}$ inductively on $j$ yeilds a reduction of $\mathfrak{u}^{\text{red}}(\mathfrak{g},S)\subset \mathfrak{u}^\prime(\mathfrak{g},S)$ of the form
$$
\mathfrak{u}^{\text{red}}(\mathfrak{g},S)=\mathfrak{g}+\bigoplus_{\text{higher weights}}\mathfrak{u}_j^{\text{red}}\subset \mathfrak{u}(\mathfrak{g}).
$$ 
Each reduction $\mathfrak{u}_j^{\text{red}}$ can be regarded as a *characteristic subspace* in $\mathfrak{g}_0\oplus \mathfrak{u}_j^\prime$, so we will refer to $\mathfrak{u}^{\text{red}}(\mathfrak{g},S)$ as the *prolongation respecting $S$ reduced by characteristic subspaces*.


### respecting multiple subspaces simultaneously

We can define the maximal prolongation $\mathfrak{u}(\mathfrak{g},S_1,\ldots,S_n)$ preserving any set $\{S_1,\ldots,S_n\}$ of destinguished subspaces similarly, or simply as the intersection 
$$
\mathfrak{u}^\prime(\mathfrak{g},S_1,\ldots,S_n):=\bigcap_{j}\mathfrak{u}^\prime(\mathfrak{g},S_j).
$$
The previous characteric supbspace reduction can be similarly defined here as well in the obvious way, yielding the *prolongation* $\mathfrak{u}^{\text{red}}(\mathfrak{g},S_1,\ldots,S_n)$ *respecting $S_1,\ldots,S_n$ reduced by characteristic subspaces*.

### Relevant `dgcv` functions
Distinguished subspaces can be specified when a `Tanaka_symbol` class object is initialized, and subsequently the `Tanaka_symbol.prolong()` method will calculate prolongations respecting the destinguished subspaces.

When respecting distinguished subspaces, `Tanaka_symbol.prolong()` defaults to compute reductions reduced by characteristic subspaces. Set the optional keyword `Tanaka_symbol.prolong(...,with_characteristic_space_reductions=False)` to forego applying reductions by characteristic subspaces (i.e., in the notation above, to compute $\mathfrak{u}^\prime(\mathfrak{g},S_1,\ldots,S_n)$ instead of $\mathfrak{u}^{\text{red}}(\mathfrak{g},S_1,\ldots,S_n)$).

Now to continue our examples, let's mark the symbol algebra from Example 2 (above) with distinguished subspaces and then prolong the symbol with this extra data.

## Continuing Example 3:

To represent subspaces in $\mathfrak{g}$, we will label a couple of lists of algebra elements that span their respective subspaces. Then we can pass these to the `distinguished_subspaces` keyword when initializing a `Tanaka_symbol` class instance, which instantiates a symbol object carying these distinguished subspaces in its internal data:

In [14]:
subspace1=[e1,e2,e6,e7,e8,e9,e10]       # Define subspaces
subspace2=[e3,e4,e5,e7,e8,e9,e10]
symbol_with_subspaces = Tanaka_symbol(g,distinguished_subspaces=[subspace1,subspace2])      # instantiate symbol object
symbol_with_subspaces.summary()     # generate table summarizing the symbol's structure

Weight,Dimension,Basis
-2,1,$e_{0}$
-1,4,"$e_{1}$, $e_{2}$, $e_{3}$, $e_{4}$"
0,6,"$e_{4}\otimes e_{1}^*$, $e_{2}\otimes e_{3}^*$, $e_{1}\otimes e_{1}^* + e_{2}\otimes e_{2}^*- e_{3}\otimes e_{3}^*- e_{4}\otimes e_{4}^*$, $2 e_{0}\otimes e_{0}^* + e_{1}\otimes e_{1}^* + e_{2}\otimes e_{2}^* + e_{3}\otimes e_{3}^* + e_{4}\otimes e_{4}^*$, $e_{1}\otimes e_{1}^*- e_{2}\otimes e_{2}^* + e_{3}\otimes e_{3}^*- e_{4}\otimes e_{4}^*$, $e_{2}\otimes e_{1}^*- e_{4}\otimes e_{3}^*$"


Now we can compute prolongations with the `.prolong()` method:

In [15]:
prolExample = symbol_with_subspaces.prolong(6,return_symbol=True,report_progress=True)

After 1st iteration:
┌────────────┬───────────────────┐
│ Weights    │ -2 │ -1 │ 0  │ 1  │
├────────────┼───────────────────┤
│ Dimensions │ 1  │ 4  │ 6  │ 4  │
└────────────┴───────────────────┘
After 2nd iteration:
┌────────────┬────────────────────────┐
│ Weights    │ -2 │ -1 │ 0  │ 1  │ 2  │
├────────────┼────────────────────────┤
│ Dimensions │ 1  │ 4  │ 6  │ 4  │ 1  │
└────────────┴────────────────────────┘
After 3rd iteration:
┌────────────┬─────────────────────────────┐
│ Weights    │ -2 │ -1 │ 0  │ 1  │ 2  │ 3  │
├────────────┼─────────────────────────────┤
│ Dimensions │ 1  │ 4  │ 6  │ 4  │ 1  │ 0  │
└────────────┴─────────────────────────────┘
After 4th iteration:
┌────────────┬──────────────────────────────────┐
│ Weights    │ -2 │ -1 │ 0  │ 1  │ 2  │ 3  │ 4  │
├────────────┼──────────────────────────────────┤
│ Dimensions │ 1  │ 4  │ 6  │ 4  │ 1  │ 0  │ 0  │
└────────────┴──────────────────────────────────┘
After 5th iteration:
┌────────────┬───────────────────────────────

In [16]:
prolExample.summary(display_length=2000)

Weight,Dimension,Basis
-2,1,$e_{0}$
-1,4,"$e_{1}$, $e_{2}$, $e_{3}$, $e_{4}$"
0,6,"$e_{4}\otimes e_{1}^*$, $e_{2}\otimes e_{3}^*$, $e_{1}\otimes e_{1}^* + e_{2}\otimes e_{2}^*- e_{3}\otimes e_{3}^*- e_{4}\otimes e_{4}^*$, $2 e_{0}\otimes e_{0}^* + e_{1}\otimes e_{1}^* + e_{2}\otimes e_{2}^* + e_{3}\otimes e_{3}^* + e_{4}\otimes e_{4}^*$, $e_{1}\otimes e_{1}^*- e_{2}\otimes e_{2}^* + e_{3}\otimes e_{3}^*- e_{4}\otimes e_{4}^*$, $e_{2}\otimes e_{1}^*- e_{4}\otimes e_{3}^*$"
1,4,"$e_{2}\otimes e_{3}^*\otimes e_{1}^* + e_{2}\otimes e_{1}^*\otimes e_{3}^*- e_{4}\otimes e_{3}^*\otimes e_{3}^*$, $- e_{2}\otimes e_{1}^*\otimes e_{1}^* + e_{4}\otimes e_{3}^*\otimes e_{1}^* + e_{4}\otimes e_{1}^*\otimes e_{3}^*$, $e_{4}\otimes e_{0}^*- e_{1}\otimes e_{1}^*\otimes e_{1}^*- e_{2}\otimes e_{2}^*\otimes e_{1}^*- e_{0}\otimes e_{0}^*\otimes e_{1}^*- e_{2}\otimes e_{1}^*\otimes e_{2}^* + e_{4}\otimes e_{3}^*\otimes e_{2}^*- e_{4}\otimes e_{1}^*\otimes e_{4}^*$, $e_{2}\otimes e_{0}^* + e_{2}\otimes e_{3}^*\otimes e_{2}^* + e_{3}\otimes e_{3}^*\otimes e_{3}^* + e_{4}\otimes e_{4}^*\otimes e_{3}^* + e_{0}\otimes e_{0}^*\otimes e_{3}^*- e_{2}\otimes e_{1}^*\otimes e_{4}^* + e_{4}\otimes e_{3}^*\otimes e_{4}^*$"
2,1,$- e_{2}\otimes e_{3}^*\otimes e_{1}^*\otimes e_{1}^*- e_{2}\otimes e_{1}^*\otimes e_{3}^*\otimes e_{1}^* + e_{4}\otimes e_{3}^*\otimes e_{3}^*\otimes e_{1}^*- e_{2}\otimes e_{1}^*\otimes e_{1}^*\otimes e_{3}^* + e_{4}\otimes e_{3}^*\otimes e_{1}^*\otimes e_{3}^* + e_{4}\otimes e_{1}^*\otimes e_{3}^*\otimes e_{3}^*$


In contrast to the standard prolongation in Example 2, this prolongation, which is preserving subspaces and *reduced by characteristic subspaces*, turned out to be finite-dimensional.

Indeed, we requested 6 prolongations, but the algorithm detected prolongations stabilization prior to reaching the 6th iteration, which is indicated by the output only displaying 5 iterations.

It is technically possible to detect the stabilization earlier for this example, but to do so one needs to observe that the symbol algebra is *fundamental*, meaning that the negative part is generated by its $-1$ level. For fundamental graded Lie algebraa (FGLA), more efficient prolongation algorithms exist, and these are automatically implemented if the FGLA property is indicated when the symbol object is initialized. One has to indicated this is by setting `assume_FGLA=True` as follows:

In [17]:
symbol_with_FGLA_property_marked = Tanaka_symbol(g,assume_FGLA=True,distinguished_subspaces=[subspace1,subspace2])
symbol_with_FGLA_property_marked.prolong(6,report_progress_and_return_nothing=True)

After 1st iteration:
┌────────────┬───────────────────┐
│ Weights    │ -2 │ -1 │ 0  │ 1  │
├────────────┼───────────────────┤
│ Dimensions │ 1  │ 4  │ 6  │ 4  │
└────────────┴───────────────────┘
After 2nd iteration:
┌────────────┬────────────────────────┐
│ Weights    │ -2 │ -1 │ 0  │ 1  │ 2  │
├────────────┼────────────────────────┤
│ Dimensions │ 1  │ 4  │ 6  │ 4  │ 1  │
└────────────┴────────────────────────┘
After 3rd iteration:
┌────────────┬─────────────────────────────┐
│ Weights    │ -2 │ -1 │ 0  │ 1  │ 2  │ 3  │
├────────────┼─────────────────────────────┤
│ Dimensions │ 1  │ 4  │ 6  │ 4  │ 1  │ 0  │
└────────────┴─────────────────────────────┘
After 4th iteration:
┌────────────┬─────────────────────────────┐
│ Weights    │ -2 │ -1 │ 0  │ 1  │ 2  │ 3  │
├────────────┼─────────────────────────────┤
│ Dimensions │ 1  │ 4  │ 6  │ 4  │ 1  │ 0  │
└────────────┴─────────────────────────────┘


Notice that this last prolongation's output stopped after 4 iterations, instead of the 5 iterations previously needed to detect stabilization. That is because this time we instantiated the symbol object with `assume_FGLA=True`. The `prolong()` method uses more efficient algorithms accordingly. Caution: if manually marking symbol objects with `assume_FGLA=True`, make sure the symbol data is actually *fundamental*, or else `prolong()` may produce unexpected results.

As described above, by default when `prolong()` computes *prolongations respecting subspaces*, if the distinguished subspaces have any non-negative-wieght elements then it further reduces the computed prolongations *by characteristic subspaces*. To override this default behavior, use `with_characteristic_space_reductions=False` (demonstrated below).

There are natural settings where one even wants the unreduced prolongations instead. Furthermore, the *reduction by characteristic subspaces* algorithm can be severely computationally demanding in high-dimensional settings, so in applications where one knows a priori that this additional reduction is trivial (i.e., yielding no actual reduction) then it is advisable to set the `with_characteristic_space_reductions=False` override for faster computation.

In [18]:
symbol_with_subspaces.prolong(4,report_progress_and_return_nothing=True,with_characteristic_space_reductions=False)

After 1st iteration:
┌────────────┬───────────────────┐
│ Weights    │ -2 │ -1 │ 0  │ 1  │
├────────────┼───────────────────┤
│ Dimensions │ 1  │ 4  │ 6  │ 6  │
└────────────┴───────────────────┘
After 2nd iteration:
┌────────────┬────────────────────────┐
│ Weights    │ -2 │ -1 │ 0  │ 1  │ 2  │
├────────────┼────────────────────────┤
│ Dimensions │ 1  │ 4  │ 6  │ 6  │ 5  │
└────────────┴────────────────────────┘
After 3rd iteration:
┌────────────┬─────────────────────────────┐
│ Weights    │ -2 │ -1 │ 0  │ 1  │ 2  │ 3  │
├────────────┼─────────────────────────────┤
│ Dimensions │ 1  │ 4  │ 6  │ 6  │ 5  │ 0  │
└────────────┴─────────────────────────────┘
After 4th iteration:
┌────────────┬──────────────────────────────────┐
│ Weights    │ -2 │ -1 │ 0  │ 1  │ 2  │ 3  │ 4  │
├────────────┼──────────────────────────────────┤
│ Dimensions │ 1  │ 4  │ 6  │ 6  │ 5  │ 0  │ 0  │
└────────────┴──────────────────────────────────┘


Notice that the positively weighted components of the last prolongation are larger than what we obtained before enforcing the `with_characteristic_space_reductions=False` option.

As mentioned in Example 2, the above symbol data is a special case of the structures from Theorem 5.3 of 

> **Porter, C. and Zelenko, I.**
> *Absolute parallelism for 2-nondegenerate CR structures via bigraded Tanaka prolongation*  
> J. reine angew. Math. (Crelles Journal) 777 (2021), 195–250

The prolongation considered there coincides with the prolongation computed above respecting `subspace1` and `subspace2` *reduced by characteristic subspaces*, i.e., computed with the default setting `with_characteristic_space_reductions=True`.

# Example 4: Getting symbol data from parabolic algebras

Parabolic subalgebras in simple algebras have natural $\mathbb{Z}$-gradings, with respect to which parabolics and their subspaces describe a large class of valid symbol data for prolongation. 

The `dgcv` library has a `simpleLieAlgebra` class representing simple Lie algebras with methods to work with some of their basic properties from classical representation theory. Namely, `simpleLieAlgebra` instances have a pre-set simple root system, and methods for extracting the graded parabolic subalgebras corresponding to subsets of simple roots.

In this example we
1. use the `createSimpleLieAlgebra` function to instantiate a representation of $\operatorname{D}_6=\mathfrak{so}(12)$,
2. use the `parabolic_subalgebra()` method to extract a graded parabolic subalgebra corresponding to a set of roots,
3. cast the graded parabolic into a `Tanaka_symbol` object, and compute its prolongations.

Let's instantiate the Lie algebra, assigning it a label `alg` to be referenced later, and assign a prefix `a` to be used in the labeling of its basis elements. (These labeling keywords are optional, and just recommended for convenience.)

In [19]:
createSimpleLieAlgebra('D6',label='alg',basis_labels='a')

When it was created, a basis of roots for the simple Lie algebra were enumerated according to standard labelings of its Dynkin diagram. Use the `root_space_summary()` method to retrieve a summary of this enumeration:

In [20]:
alg.root_space_summary()

This simple algebra D6 has 6 roots (r_1, ..., r_6), which are dual to the Cartan subalgebra basis (a_1, a_2, a_3, a_4, a_5, a_6). These roots correspond to vertices in the Dynkin diagram as follows:

┌───────────────────────┐
│          D6           │
╞═══════════════════════╡
│   r_1 r_2 r_3 r_4 r_5 │
│   ◯───◯───◯───◯───◯   │
│               │       │
│               ◯ r_6   │
└───────────────────────┘


For this example, let's consider the parabolic corresponding to the roots 2, 4, 5, and 6. The `parabolic_subalgebra` method will return this subalgebra with its natural grading. By defualt the returned parabolic is assigned a non-negative grading, but since we will cast it to a `Tanaka_symbol` class, we rather need the opposite sign convention. To flip the sign convention, we can use `use_non_positive_weights=True`.

In the following command, we are creating a new object, and to reference it later we should assign it a label. We will assign it the label `'parabolic_alg'` and let's label the its basis labels `'alpha'`:

In [21]:
alg.parabolic_subalgebra([2,4,5,6],label='parabolic_alg',basis_labels='alpha',use_non_positive_weights=True)

As we have now instantiated many objects, the `vmf_summary()` function is a useful tool to get a quick overview of the instantiated objects. The acronym *VMF* refers to `dgcv`'s variable management framework, which keeps track of instantiated mathematical objects, and `vmf_summary()` displays which objects are currently in the scope of the VMF.

For algebras in the VMF, the summary table will display useful properties like their dimensions and weights of their assigned basis vectors w.r.t. various assigned gradings.

In [22]:
vmf_summary()

Coordinate System,# of Variables,Real Part,Imaginary Part,Vector Fields,Differential Forms
"$a = \left( a_{0}, \ldots, a_{10} \right)$",11,----,----,"$\frac{\partial}{\partial a_{0}}$, $\ldots$, $\frac{\partial}{\partial a_{10}}$","$\operatorname{d} a_{0}$, $\ldots$, $\operatorname{d} a_{10}$"
"$x = \left( x_{1}, \ldots, x_{5} \right)$",5,----,----,"$\frac{\partial}{\partial x_{1}}$, $\ldots$, $\frac{\partial}{\partial x_{5}}$","$\operatorname{d} x_{1}$, $\ldots$, $\operatorname{d} x_{5}$"


Algebra Label,Basis,Dimension,Grading
$\mathfrak{alg}$,"$a_{1}$, ..., $a_{66}$",66,"(0, 0, 0, 0, 0, 0, -1, -1, -1, -1, -1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, -1, -1, -1, 0, 1, -1, -1, -1, 0, 1, 1, -1, -1, 0, 1, 1, 1, -1, 0, 1, 1, 1, 1), (0, 0, 0, 0, 0, 0, 0, -1, -1, -1, -1, 0, -1, -1, -1, -1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0, -1, -1, -1, 0, 0, 1, -1, -1, 0, 0, 1, 1, -1, 0, 0, 1, 1, 1), (0, 0, 0, 0, 0, 0, 0, 0, -1, -1, -1, 0, 0, -1, -1, -1, 0, 0, -1, -1, -1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, -1, 1, 0, 0, 0, -1, -1, 0, 0, 0, 0, 0, 0, -1, -1, 0, 0, 0, 1, -1, 0, 0, 0, 1, 1), (0, 0, 0, 0, 0, 0, 0, 0, 0, -1, -1, 0, 0, 0, -1, -1, 0, 0, 0, -1, -1, 0, 0, 0, -1, -1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, -1, 1, 1, 0, 0, -1, -1, 1, 0, 0, -1, -1, -1, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0, 1), (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0, -1, 0, 0, 0, 0, -1, 0, 0, 0, 0, -1, 0, 0, 0, 0, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, -1, 1, 1, 1, 0, -1, -1, 1, 1, 0, -1, -1, -1, 1, 0, -1, -1, -1, -1, 0, 0, 0, 0, 0, 0), (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, -1, -1, 1, 1, 1, -1, -1, -1, 1, 1, -1, -1, -1, -1, 1, -1, -1, -1, -1, -1)"
$\mathfrak{g}$,"$e_{0}$, ..., $e_{10}$",11,"(-2, -1, -1, -1, -1, 0, 0, 0, 0, 0, 0)"
$\mathfrak{parabolic}_{alg}$,"$\alpha_{1}$, ..., $\alpha_{40}$",40,"(0, 0, 0, 0, 0, 0, 0, -1, -1, -2, -3, 0, -1, -1, -2, -3, 0, -1, -2, 0, -1, -2, -1, -4, -3, -3, 0, -3, -3, -2, 0, -2, -2, -1, -1, -1, -1, -1, 0, 0)"
$\mathfrak{prolongation}_{algebra}$,"$\eta_{1}$, ..., $\eta_{14}$",14,"(-3, -3, -2, -1, -1, 0, 0, 0, 0, 1, 1, 2, 3, 3)"


Now let's cast the parabolic algebra to a `Tanaka_symbol` class. The initial casting by this method can be a bit slow for high-dimensional algebras due to necessary reformatting of non-negative parts in the symbol. 

In [23]:
g=Tanaka_symbol(parabolic_alg)
g.summary()

Weight,Dimension,Basis
-4,1,$\alpha_{24}$
-3,6,"$\alpha_{11}$, $\alpha_{16}$, $\alpha_{25}$, $\alpha_{26}$, $\alpha_{28}$, $\alpha_{29}$"
-2,7,"$\alpha_{10}$, $\alpha_{15}$, $\alpha_{19}$, $\alpha_{22}$, $\alpha_{30}$, $\alpha_{32}$, $\alpha_{33}$"
-1,12,"$\alpha_{8}$, $\alpha_{9}$, $\alpha_{13}$, $\alpha_{14}$, $\alpha_{18}$, $\alpha_{21}$, $\alpha_{23}$, $\alpha_{34}$, $\alpha_{35}$, $\alpha_{36}$, $\alpha_{37}$, $\alpha_{38}$"
0,14,output too long to display; raise `display_length` to a higher bound if needed.


Now we are ready to prolong!

In [24]:
g_prolong = g.prolong(5,return_symbol=True,report_progress=True)
g_prolong.summary()

After 1st iteration:
┌────────────┬─────────────────────────────┐
│ Weights    │ -4 │ -3 │ -2 │ -1 │ 0  │ 1  │
├────────────┼─────────────────────────────┤
│ Dimensions │ 1  │ 6  │ 7  │ 12 │ 14 │ 12 │
└────────────┴─────────────────────────────┘
After 2nd iteration:
┌────────────┬──────────────────────────────────┐
│ Weights    │ -4 │ -3 │ -2 │ -1 │ 0  │ 1  │ 2  │
├────────────┼──────────────────────────────────┤
│ Dimensions │ 1  │ 6  │ 7  │ 12 │ 14 │ 12 │ 7  │
└────────────┴──────────────────────────────────┘
After 3rd iteration:
┌────────────┬───────────────────────────────────────┐
│ Weights    │ -4 │ -3 │ -2 │ -1 │ 0  │ 1  │ 2  │ 3  │
├────────────┼───────────────────────────────────────┤
│ Dimensions │ 1  │ 6  │ 7  │ 12 │ 14 │ 12 │ 7  │ 6  │
└────────────┴───────────────────────────────────────┘
After 4th iteration:
┌────────────┬────────────────────────────────────────────┐
│ Weights    │ -4 │ -3 │ -2 │ -1 │ 0  │ 1  │ 2  │ 3  │ 4  │
├────────────┼────────────────────────────────

Weight,Dimension,Basis
-4,1,$\alpha_{24}$
-3,6,"$\alpha_{11}$, $\alpha_{16}$, $\alpha_{25}$, $\alpha_{26}$, $\alpha_{28}$, $\alpha_{29}$"
-2,7,"$\alpha_{10}$, $\alpha_{15}$, $\alpha_{19}$, $\alpha_{22}$, $\alpha_{30}$, $\alpha_{32}$, $\alpha_{33}$"
-1,12,"$\alpha_{8}$, $\alpha_{9}$, $\alpha_{13}$, $\alpha_{14}$, $\alpha_{18}$, $\alpha_{21}$, $\alpha_{23}$, $\alpha_{34}$, $\alpha_{35}$, $\alpha_{36}$, $\alpha_{37}$, $\alpha_{38}$"
0,14,output too long to display; raise `display_length` to a higher bound if needed.
1,12,output too long to display; raise `display_length` to a higher bound if needed.
2,7,output too long to display; raise `display_length` to a higher bound if needed.
3,6,output too long to display; raise `display_length` to a higher bound if needed.
4,1,output too long to display; raise `display_length` to a higher bound if needed.


Counting dimensions, one sees that this prolongation reproduced the full $\mathfrak{so}(12)$.

# Example 5: Prolonging subspaces

Let's continue from Example 4, so, in particular, make sure `parabolic_alg` from Example 4 is initialized with its basis ($\alpha_1,\ldots, \alpha_{40}$).

Passing a list of algebra elements to the `algebra_class`'s `subalgebra` or `subspace` methods will return a subalgebra or subspace object (either `algebra_subspace_class` or `subalgebra_class` type). If the given spanning elements are weighted homogeneous with respect to the parent algebra's marked gradings then the subalgebra/subspace will also be marked with corresponding gradings automatically. A `algebra_subspace_class` or `subalgebra_class` instance marked with appriate $\mathbb{Z}$-gradings can be cast into a `Tanaka_symbol` object. Here is an example:

In [25]:
subspace_basis = [alpha_8,alpha_18,alpha_34,alpha_10,alpha_32,alpha_25]
subalgebra_example = parabolic_alg.subalgebra(subspace_basis)
symbol_example = Tanaka_symbol(subalgebra_example)
symbol_example.summary()

Weight,Dimension,Basis
-3,1,$\alpha_{25}$
-2,2,"$\alpha_{10}$, $\alpha_{32}$"
-1,3,"$\alpha_{8}$, $\alpha_{18}$, $\alpha_{34}$"


Now we can compute its prolongations with the `prolong` method. 

In [26]:
prolongation_example=symbol_example.prolong(6,return_symbol=True)
prolongation_example.summary()

Weight,Dimension,Basis
-3,1,$\alpha_{25}$
-2,2,"$\alpha_{10}$, $\alpha_{32}$"
-1,3,"$\alpha_{8}$, $\alpha_{18}$, $\alpha_{34}$"
0,3,"$\alpha_{25}\otimes \alpha_{25}^* + \alpha_{10}\otimes \alpha_{10}^* + \alpha_{32}\otimes \alpha_{32}^* + \alpha_{8}\otimes \alpha_{8}^*$, $\alpha_{25}\otimes \alpha_{25}^* + \alpha_{10}\otimes \alpha_{10}^* + \alpha_{18}\otimes \alpha_{18}^*$, $\alpha_{25}\otimes \alpha_{25}^* + \alpha_{32}\otimes \alpha_{32}^* + \alpha_{34}\otimes \alpha_{34}^*$"
1,3,output too long to display; raise `display_length` to a higher bound if needed.
2,2,output too long to display; raise `display_length` to a higher bound if needed.
3,1,output too long to display; raise `display_length` to a higher bound if needed.


With the `createAlgebra` function, we can extract the abstract algebra structure from this prolongation and use `algebra_class` methods to study its properties:

In [27]:
createAlgebra(prolongation_example,label='prol_example_alg',basis_labels='b')
prol_example_alg.is_semisimple(verbose=True,return_bool=False)

prol_example_alg is skew-symmetric.
prol_example_alg satisfies the Jacobi identity.
prol_example_alg is a Lie algebra.
Progress update: computing determinant of the Killing form...
prol_example_alg is semisimple.


This last prolongation is isomorphic to $\mathfrak{sl}(4,\mathbb{C})$.